### 1. 영어 Word2Vec 만들기
- Gensim 활용

In [1]:
import re
import urllib.request
import zipfile
from lxml import etree
from nltk.tokenize import word_tokenize, sent_tokenize

1) 훈련 데이터 다운로드

In [2]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/09.%20Word%20Embedding/dataset/ted_en-20160408.xml", filename="ted_en-20160408.xml")

('ted_en-20160408.xml', <http.client.HTTPMessage at 0x289c28eb400>)

2. 훈련 데이터 전처리하기

In [5]:
target_xml = open('ted_en-20160408.xml', 'r', encoding='UTF8')
target_text = etree.parse(target_xml)

# xml 파일로부터 <content> 사이의 내용만 가져온다
parse_text = '\n'.join(target_text.xpath('//content/text()'))

# 정규 표현식으로 배경음 제거
content_text = re.sub(r'\([^)]*\)', '', parse_text)

# 입력 코퍼스를 토큰화
sent_text = sent_tokenize(content_text)

# 각 문장 구두점 제거 및 소문자화
normalized_text = []
for ss in sent_text:
    tokens = re.sub(r"[^a-z0-9]+", " ", ss.lower())
    normalized_text.append(tokens)

# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행
result = [word_tokenize(sentence) for sentence in normalized_text]

In [6]:
print("샘플의 개수 : {}".format(len(result)))

샘플의 개수 : 273380


3. Word2Vec 훈련시키기
- size : 워드 벡터의 특징 값 , 임베딩 된 벡터의 차원
- window : context 윈도우 크기
- min_count = 단어 최소 빈도 수 제한(빈도가 적은 단어들을 학습하지 않음)
- workers = 학습을 위한 프로세스의 수
- sg : 0은 CBOW, 1은 Skip-gram


In [1]:
from gensim.models import Word2Vec, KeyedVectors

model = Word2Vec(sentences=result, size=100, window=5, min_count=5, workers=4, sg=0)

ValueError: numpy.ndarray size changed, may indicate binary incompatibility. Expected 88 from C header, got 80 from PyObject

In [ ]:
model_result = model.wv.most_similar("man")

4. Word2Vec 모델 저장 & 로드 하기

In [ ]:
model.wv.save_word2vec_format('eng_w2v')
loaded_model = KeyedVectors.load_word2vec_format("eng_w2v")

In [ ]:
model_result = loaded_model.most_similar("man")
print(model_result)

# 2) 한국어 Word2Vec 만들기

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
from gensim.models.word2vec import Word2Vec
from konlpy.tag import Okt

1. 데이터 로드

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")

2. 데이터 전처리

In [ ]:
train_data = pd.read_table('ratings.txt')
train_data[:5]

In [ ]:
# 결측값 처리
train_data = train_data.dropna(how='any')

# 정규표현식으로 한글 외 문자 제거
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣]","")

# 불용어 처리
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

# 형태소 분석기로 토큰화 작업
okt = Okt()

tokenized_data = []
for sentence in tqdm(train_data['document']):
    tokenized_sentence = okt.morphs(sentence, stem=True)
    stopword_removed = [word for word in tokenized_sentence if not word in stopwords]
    tokenized_data.append(stopword_removed)
    

In [ ]:
# 리뷰 길이 분포 확인
print('리뷰의 최대 길이 :',max(len(review) for review in tokenized_data))
print('리뷰의 평균 길이 :',sum(map(len, tokenized_data))/len(tokenized_data))
plt.hist([len(review) for review in tokenized_data], bins=50)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

3. 네이버 리뷰 학습

In [ ]:
from gensim.models import Word2Vec

model = Word2Vec(sentences = tokenized_data, size = 100, window = 5, min_count = 5, workers = 4, sg = 0)

In [ ]:
# 완성된 임베딩 매트릭스의 크기 확인
model.wv.vectors.shape

In [ ]:
print(model.wv.most_similar("최민식"))

# 3) 사전 훈련된 Word2Vec 임베딩
- 감성 분류 작업할 때 데이터양이 부족한 상태라면, 사전 학습 임베딩 벡터를 가지고 온다.
- 

In [ ]:
import gensim
import urllib.request

# 구글의 사전 훈련된 Word2Vec 모델을 로드.
urllib.request.urlretrieve("https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz", \
                           filename="GoogleNews-vectors-negative300.bin.gz")
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)

In [ ]:
print(word2vec_model.vectors.shape)

In [ ]:
# 2 단어의 유사도 확인하기
print(word2vec_model.similarity('this', 'is'))
print(word2vec_model.similarity('post', 'book'))